# Sistemas numéricos de punto flotante y error numérico

## Sistemas numéricos

En matemáticas, tenemos varias formas de representar a un número. Por ejemplo, las expresiones

$$ \frac{17}{17}, \quad 1.0\times 10^0, \quad 0.\overline{999}, \quad \lim_{x\to -1} x^2, \quad \sum_{n=1}^\infty \frac{1}{(n(n+1))}, \quad -e^{i\pi}, \quad \text{etcétera}$$

representan al número 1. Sin embargo, al momento de hacer cálculos numéricos resulta óptimo fijar una forma de representar a todos los números con los que queremos trabajar, lo que se conoce como un _sistema numérico_, y procurar que éste sea ventajoso para el tipo de cálculos que queremos hacer.

## Sistemas numéricos de punto flotante

En muchas ramas científicas es necesario poder manipular números reales (i.e., elementos de $\mathbb{R}$) de forma eficiente. Sin embargo, dado que la capacidad de almacenamiento de las computadoras es limitada, muchos números no se pueden representar de manera _fiel_  (es decir, sin pérdida de información) utilizando una computadora. En particular, la forma más natural de operar con números en una computadora es representándolos como secuencias de _bits_ (ceros y unos). En efecto, las operaciones que las computadoras realizan a nivél físico (es decir, electrónico), es entre valores binarios (cero/uno, encendido/apagado, verdadero/falso), utilizando transistores y circuitos.

### Error de redondeo

Los números enteros (i.e., elementos de $\mathbb{Z}$) generalmente pueden ser representados de manera fiel como una serie de bits -siempre que se tenga suficiente memoria disponible. Entre más bits se necesiten para almacenar un entero, _más se tardará la computadora en realizar operaciones con él_. Entonces, mientras se tengan suficiente _bits_, representar fielmente a los números enteros y operar con ellos será _plausible_, y el único tipo de error que deberemos buscar evitar es el **sobreflujo**.

Sin embargo, ¿qué sucede con los números que tienen decimales no nulos? Este tipo de números también son almacenados como secuencias de bits, pero ahora algunos de esos bits representarán potencias _negativas_ de $2$ (es decir, $\frac{1}{2}, \frac{1}{4}, \frac{1}{8},$ etc.). Veamos qué consecuencias puede tener esto cuando nos enfrentamos a la memoria limitada de una computadora.

Consideremos al número $\frac{1}{10}$ ("un décimo"), que se ve como $0.1$ en sistema decimal. Resulta que $\frac{1}{10}$ se expresa como suma de potencias de dos de la siguiente forma:

$$0.1 = 2^{-4} + 2^{-5} + 2^{-8} + 2^{-9} + 2^{-12} + 2^{-13} + ... = \sum_{n=1}^\infty \big( 2^{-(4n)} + 2^{-(4n+1)} \big)$$

es decir, se obtiene a través de una serie infinita de potencias de dos. En otras palabras, este número que se escribe con una sola cifra decimal en el sistema decimal _¡se representa con una **infinidad** de cifras en el sistema binario!_

$$\frac{1}{10} = 0.00011001100110011\dots$$

Dado que la computadora no puede almacenar _una infinidad de bits_, sólo podrá representar al número $\frac{1}{10}$ _de manera aproximada_. Esto se conoce como **error de redondeo**; el _valor redondeado_ es la mayor aproximación posible que puede hacer la computadora con la cantidad de bits utilizados.

Este problema no es cosa menor: la _mayoría_ de los números reales (i.e., elementos de $\mathbb{R}$) se expresan como series **infinitas** de potencias de $2$ por lo que, al trabajar con una computadora, _la mayoría de los números reales no podrán ser representados fielmente, sino que sufrirán un error de redondeo_!

A veces este error no es evidente a simple vista:

In [ ]:
0.1

Sin embargo, al operar entre números que tienen error de redondeo, estos errores **se amplifican**:

In [ ]:
0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1

Por lo tanto, para hacer cómputo científico, es **crucial** estar conscientes del error de redondeo que tendrán los números con los que operemos. Utilizar una **mayor cantidad de bits** para representar números reales nos dará una **mayor precisión**, pero vendrá con el costo de un **mayor tiempo de cómputo**. Por lo tanto, para un proyecto de cómputo científico, se deben hacer elecciones entre la precisión que se requiere para los cálculos y el tiempo que se tiene disponible para hacer los cálculos.

### `Float32` y `Float64`

En Julia, los principales tipos numéricos primitivos para números de punto flotante son `Float32` y `Float64`, los cuales utilizan 32 y 64 bits, respectivamente, para representar a los números literales. De forma predeterminada, Julia considera a los números literales escritos con punto decimal o notación científica (usando `e`) como flotantes de 32 ó 64 bits:

In [ ]:
typeof(1.0)

In [ ]:
typeof(2.99792458e8) # Velocidad de la luz en m/s

mientras que, si escribimos un número literal en notación científica usando `f` en vez de `e`, Julia lo considera como flotante de 32 bits:

In [ ]:
typeof(2.99702458f8)

Para convertir entre estos tipos numéricos flotantes, podemos llamar a `Float32` y `Float64` como funciones.

In [ ]:
typeof( Float32(2.99792458e8) ) # Cambia de Float64 a Float32

In [ ]:
typeof( Float64(2.99792458f8) ) # Cambia de Float32 a Float64

Por completez, mencionamos que en Julia también existe un tipo numérico primitivo para números de punto flotante de 16 bits llamado`Float16`, y que podemos llamar a `Float16` como función para convertir a números de de este tipo; sin embargo, no lo utilizaremos en este curso.

Para un ser humano, más _bits_ implica una **mayor precisión**, es decir, una mayor cercanía entre los números que _queremos representar_ y los que una computadora _puede usar para representarlos_. Sin embargo, para una computadora, más _bits_ implica _más información con la cual trabajar_, lo que a su vez implica un **mayor tiempo de cómputo**. En particular, las computadoras ven como números "grandes" a aquellos que necesiten muchos bits para ser representados, ¡los cuales pueden tener valores cercanos a infinito ("grandes" para un ser humano) _o a **cero**_ ("pequeños")!

Para una lista completa de los tipos numéricos primitivos en Julia, puedes consultar la [documentación](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Integers-and-Floating-Point-Numbers).

## Épsilon de máquina

Como vimos en la sección pasada, la representación de una gran cantidad de números (por ejemplo, los recursivos y los trascendentales) en un sistema de punto flotante conlleva un proceso de redondeo. Esto implica la existencia de un error relativo en el redondeo, el cual se calcula como

$$\text{error relativo} = \frac{\text{error absoluto}}{\text{valor redondeado,}}$$

donde $\text{error absoluto}$ es la diferencia entre el valor que se quería representar y el $\text{valor}$ (al cual fue) $\text{redondeado}$.

A través de argumentos heurísticos, es posible ver que los errores relativos más grandes en un sistema de punto flotante ocurren entre los números _mayores_ que uno _que son redondeados a uno_, y que estos aumentan a medida que aumenta el número. Por lo tanto, se considera como **cota superior del error relativo del sistema de punto flotante** al _mayor número que el sistema redondea a uno_, y se le llama **épsilon de máquina**.

**Ejercicio** Implementa un código que aproxime la épsilon de máquina para  `Float32` y `Float64` (Sugerencia: Utiliza un ciclo _while_).

In [ ]:
# Tu código comentado va aquí :D

### `eps`

En Julia, la función `eps` nos devuelve la épsilon de máquina de un tipo numérico de punto flotante:

In [ ]:
eps(Float32)

In [ ]:
eps(Float64)

Si como entrada le damos un número de punto flotante de cierto tipo, `eps` nos devuelve el siguiente número de punto flotante del mismo tipo que puede ser representado:

In [ ]:
eps(1.0) # Observen que toma a 1.0 como de tipo Float64

## Propagación de error

Como cualquier persona que esté familiarizada con física experimental sabe, ninguna medición física está exenta de error y, al hacer cálculos con datos tomados a partir de mediciones físicas, es importante considerar cómo es que los errores de las mediciones se propagan al cálculo final; suponer que al hacer operaciones con números inciertos podemos obtener un resultado exacto sería **absurdo**. Una fórmula general para calcular el error absoluto de una función de varias variables $f(x_1,...,x_n)$ propagado por las variables $x_1,...,x_n$ con error absoluto $\Delta x_1,...,\Delta x_n$ es[3]

$$\Delta f = \sqrt{ \sum_{i=1}^n \bigg(\frac{\partial f}{\partial x_i} \Delta x_i\bigg)^2 },$$

y está acotado superiormente por

$$\Delta f \leq \sum_{i=1}^n \bigg| \frac{\partial f}{\partial x_i} \bigg| \Delta x_i.$$

Dado que las computadoras no escapan el mundo físico (aunque a veces parezca que nos ayudan a hacerlo) es crucial que, siempre que hagamos cálculos numéricos **de alta precisión**, tomemos en cuenta las **limitaciones de los sistemas numéricos** implementados en las computadoras que utilicemos.

## Resumen

Un _sistema numérico_ es una forma de representar números, necesario para hacer cálculos numéricos. Los sistemas numéricos más comunmente utilizados para cálculos en computadoras son los _sistemas de punto flotante_, pues tienen las bondades de ser fáciles de implementar (sobre todo los de base 2), usar, y estandarizar, además de ser rápidos, versátiles (útiles para trabajar tanto con números muy grandes como muy pequeños), y precisos. Sin embargo, dado que la capacidad de memoria de las computadoras es limitada, ocurren errores de redondeo al representar a algunos números en este tipo de sistemas. Por lo tanto, al hacer cálculos de alta precisión, es importante tener en cuenta la _épsilon de máquina_ (cota superior del error relativo en un sistema de punto flotante) y la _propagación de errores_ en nuestro cálculos, así como otras limitaciones de nuestros instrumentos de cálculo (nuestras computadoras).

En Julia, los principales tipos numéricos primitivos para números de punto flotante son `Float32` y `Float64`, que para representar números usan 32 y 64 _bits_, respectivamente, y la épsilon de máquina para cada tipo se puede obtener con la función `eps`. En general, aumentar la cantidad de _bits_ utilizados para representar números reducirá los errores de redondeo a cambio de aumentar el tiempo de cómputo, y vice versa.

## Dato curioso

Julia también implementa soporte para [aritmética de precisión arbitraria](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Arbitrary-Precision-Arithmetic), aunque estos cálculos son relativamente más lentos.

## Bibliografía

[1] Muller et al, _Handbook of Floating-Point Arithmetic_ (2018). 

[2]
Baird, _Experimentación: Una introducción a la teoría de mediciones y al diseño experimental_ (1991).

[3] Taylor, _An Introduction to Error Analysis: The Study of Uncertanties in Physical Measurements_ (1982).

## Recursos complementarios
* Video [_Floating Point Numbers_](https://www.youtube.com/watch?v=PZRI1IfStY0) del canal de YouTube Computerphile.
* [Documentación de números enteros y de punto flotante en Julia](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/).
* Sección 1.2 "Round-off Errors and Computer Arithmetic" de Burden et al, _Numerical Analysis_ (2019).